In [90]:
import pandas as pd
from tqdm import tqdm
import os
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

In [8]:
print(os.getcwd())

c:\qtri\Self-learning\Spotify Recommendation\Spotify_Recommendation


### Data Import

In [9]:
df = pd.read_csv("data/processed_data (1).csv")

In [19]:
df.columns

Index(['pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name',
       'album_uri', 'duration_ms_x', 'album_name', 'name', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri',
       'track_href', 'analysis_url', 'duration_ms_y', 'time_signature',
       'artist_pop', 'genres', 'track_pop'],
      dtype='object')

### Preprocessing
1. Useful datat selection (dedup and drop first 2 columns)
2. Bring uris to its basic format (artist_uri, album_uri, uri) (but we don't use them tho)
3. List concatenation for genres

In [17]:
df.drop(['Unnamed: 0.1', 'Unnamed: 0', 'uri', 'id'], axis = 1, inplace = True)

In [20]:
df.columns

Index(['pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name',
       'album_uri', 'duration_ms_x', 'album_name', 'name', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri',
       'track_href', 'analysis_url', 'duration_ms_y', 'time_signature',
       'artist_pop', 'genres', 'track_pop'],
      dtype='object')

In [21]:
def dedup(df):
    df['artist_song'] = df.apply(lambda row: row['artist_name']+row['track_name'], axis = 1)
    return df.drop_duplicates('artist_song')

songDF = dedup(df)

In [33]:
#but we don't need to use these uris thou
songDF['track_uri'] = songDF['track_uri'].apply(lambda x: re.findall(r'\w+$', x)[0])
songDF['album_uri'] = songDF['album_uri'].apply(lambda x: re.findall(r'\w+$', x)[0])

C:\Users\voqua\AppData\Local\Temp\ipykernel_69076\1638886571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songDF['track_uri'] = songDF['track_uri'].apply(lambda x: re.findall(r'\w+$', x)[0])
C:\Users\voqua\AppData\Local\Temp\ipykernel_69076\1638886571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songDF['album_uri'] = songDF['album_uri'].apply(lambda x: re.findall(r'\w+$', x)[0])


In [35]:
def select_cols(df):
       '''
       Select useful columns
       '''
       return df[['artist_name','track_name','danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', "artist_pop", "genres", "track_pop"]]
songDF = select_cols(songDF)

List concatenation.

In [36]:
songDF['genres'] = songDF['genres'].apply(lambda genre: genre.split(" "))

### Feature Generation / Feature Engineer
1. One-hot encoding
2. Sentiment Analysis
3. TF-IDF
4. Normalization

In [70]:
# OHE 
# this only applies to categorical columns with one value in each datapoint, won't work for attribute with several values
# this doesn't add weights to different categories, so every option weights the same
def ohe(df, column_name, new_name):
    ohe_df = pd.get_dummies(df[column_name])
    feature_name = ohe_df.columns
    new_name = [ str(new_name) + " | " + str(i) for i in feature_name]
    ohe_df.columns = new_name
    return ohe_df

In [86]:
#TF-IDF for genres
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(songDF['genres'].apply(lambda x: " ".join(x))) #fit_transform inputs a sentence of genres seperated by a whitespcae
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + " | " + i for i in tfidf.get_feature_names_out()]
genre_df.drop(columns='genre | unknown') # Drop unknown genre, specified by a word "unknown"
genre_df.reset_index(drop = True, inplace=True)

In [104]:
#Normalization for track_pop
pop = songDF[["track_pop"]].reset_index(drop = True)
scaler = MinMaxScaler()
pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = ['track_pop'])

In [108]:
songDF.columns

Index(['artist_name', 'track_name', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'artist_pop', 'genres', 'track_pop'],
      dtype='object')

In [114]:
# One-hot Encoding
key_ohe = ohe(df, 'key','key') * 0.5
mode_ohe = ohe(df, 'mode','mode') * 0.5

In [124]:
songDF['tempo'].values

array([125.461, 143.04 ,  99.259, ..., 140.089, 121.633, 117.583])

In [135]:
# Scale tempo column
scale = MinMaxScaler()
tempo_scaled = pd.DataFrame(scale.fit_transform(songDF[['tempo']]), columns = ['tempo_scaled'])

In [ ]:
#Sentiment Analysis. Conduct with track name to find polarity and subjectivity.